# Identify impact hotspots in African countries

* **Products used:** [DE Africa Coastlines]()



### Load packages
Import Python packages that are used for the analysis.

In [1]:
import geopandas as gpd
import pandas as pd
import rioxarray as rxr
import numpy as np
from deafrica_tools.spatial import xr_rasterize, xr_vectorize

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


In [2]:
version = 'v0.4.2'
coastlines = gpd.read_file(f"../data/deafricacoastlines_{version}_gadm_level1.gpkg")

In [ ]:
coastlines = coastlines[(coastlines.certainty=='good') & (coastlines.sig_time<0.01) & (coastlines.rate_time<-0.5)]
admins = coastlines['GID_1'].unique()

In [23]:
len(admins)

4

In [24]:
import os
import datacube

os.environ['AWS_DEFAULT_REGION']="us-east-1"
os.environ['AWS_S3_ENDPOINT']="s3.us-east-1.amazonaws.com"

# get hotspots and population
# Load population data
ds = rxr.open_rasterio('s3://dataforgood-fb-data/hrsl-cogs/hrsl_general/hrsl_general-latest.vrt', 
                       chunks ={'x': 1000, 'y': 1000})
ds = ds.rename({'x':'longitude','y':'latitude'})


dc=datacube.Datacube()
mrvbf = dc.load(product="dem_srtm_deriv", measurements=["mrvbf"], dask_chunks={'longitude':2000, 'latitude':2000}, 
                like=ds.geobox).mrvbf.squeeze()

In [80]:
for admin in admins:
    if admin == '':continue
    print(admin)
    
    buffered = coastlines[coastlines['GID_1']==admin].copy()
    buffer=500
    buffered['geometry'] = buffered.to_crs('6933').buffer(buffer).to_crs('4326')
    
    coast_bbox = buffered.unary_union.bounds
    
    os.environ['AWS_DEFAULT_REGION']="us-east-1"
    os.environ['AWS_S3_ENDPOINT']="s3.us-east-1.amazonaws.com"
    pop_raster = ds.isel(band=0).sel(longitude=slice(coast_bbox[0],coast_bbox[2]), latitude = slice(coast_bbox[3], coast_bbox[1])).compute()#.astype(int)
    print(pop_raster.shape)
    
    
    os.environ['AWS_DEFAULT_REGION']="af-south-1"
    os.environ['AWS_S3_ENDPOINT']="s3.af-south-1.amazonaws.com"
    mrvbf_raster = mrvbf.sel(longitude=slice(coast_bbox[0],coast_bbox[2]), latitude = slice(coast_bbox[3], coast_bbox[1])).compute()

    hotspots_mask = xr_rasterize(gdf=buffered,
                                da=pop_raster,
                                  transform=pop_raster.geobox.transform,
                                  crs=pop_raster.geobox.crs)
    hotspots_map = pop_raster*(mrvbf_raster==7)*hotspots_mask
    
    nhotspots = (hotspots_map>0).sum()
    if nhotspots>0:
        print("# of hotspot pixels", nhotspots)
        risk_vectors = xr_vectorize(hotspots_map, mask=hotspots_map>0, attribute_col='pop', crs='epsg:4326', export_shp=f'coastchange_hotspots/{admin}_hotspots.geojson')
        break
    else:
        print("not hotspots")
        
    del pop_raster, mrvbf_raster, hotspots_mask, hotspots_map

GHA15_2
(634, 2906)
not hotspots
GHA5_2
(1455, 3961)
not hotspots
GHA7_2
(1132, 3890)
# of hotspot pixels <xarray.DataArray ()>
array(1057)
Coordinates:
    band         int64 1
    time         datetime64[ns] 2000-01-01T12:00:00
    spatial_ref  int32 4326
